In [3]:
# Import tinydb
from tinydb import TinyDB, Query

In [4]:
# create a db and a corresponding query
db = TinyDB('more-users.json')
User = Query()

In [5]:
# Add some users
db.insert({'name': 'John', 'age': 65})
db.all()

[{'name': 'John', 'age': 65}]

In [6]:
# Checking for the existence of fields
db.search(User.name.exists())

[{'name': 'John', 'age': 65}]

In [7]:
User.name.exists()

QueryImpl('exists', ('name',))

In [8]:
db.search(User.name.matches('[aZ]*'))

[{'name': 'John', 'age': 65}]

In [9]:
import re

# Case insensitive search for John
db.search(User.name.matches('John', flags=re.IGNORECASE))

[{'name': 'John', 'age': 65}]

In [10]:
# Any part of the item has to match the regex
db.search(User.name.matches('b+'))

[]

In [11]:
# We can also find the item if it passes a custom test
test_func = lambda s: s == 'John'
db.search(User.name.test(test_func))

[{'name': 'John', 'age': 65}]

In [12]:
# Custom test with parameters
def test_func(val, m, n):
    return m <= val <= n

db.search(User.age.test(test_func, 0, 21))

[]

In [13]:
db.search(User.age.test(test_func, 21, 99))

[{'name': 'John', 'age': 65}]

In [14]:
# when a field contains a list
db2 = TinyDB('usersgroups.json')

In [16]:
db2.insert({'name': 'user1', 'groups': ['user']})
db2.insert({'name': 'user2', 'groups': ['user', 'admin']})
db2.insert({'name': 'user3', 'groups': ['user', 'sudo']})

3

In [17]:
# Using any and all
db2.search(Query().groups.any(['sudo', 'admin']))

[{'name': 'user2', 'groups': ['user', 'admin']},
 {'name': 'user3', 'groups': ['user', 'sudo']}]

In [18]:
db2.search(Query().groups.all(['user', 'admin']))

[{'name': 'user2', 'groups': ['user', 'admin']}]

In [19]:
# Nested Queries
Group = Query()
Permission = Query()

In [21]:
db3 = TinyDB('groups.json')

In [23]:
groups = db3.table('groups')

In [26]:
groups.insert({
    'name': 'user',
    'permissions': [{'type': 'read'}]
})

2

In [27]:
groups.insert({
    'name': 'sudo',
    'permissions': [{'type': 'read'}, {'type': 'sudo'}]
})

groups.insert({
    'name': 'admin',
    'permissions': [{'type': 'read'}, {'type': 'write'}, {'type': 'sudo'}]
})

4

In [28]:
# Search group has a permission with type read
groups.search(Group.permissions.any(Permission.type == 'read'))

[{'name': 'user', 'permissions': [{'type': 'read'}]},
 {'name': 'sudo', 'permissions': [{'type': 'read'}, {'type': 'sudo'}]},
 {'name': 'admin',
  'permissions': [{'type': 'read'}, {'type': 'write'}, {'type': 'sudo'}]}]

In [29]:
# Group with only READ permission
groups.search(Group.permissions.all(Permission.type == 'read'))

[{'name': 'user', 'permissions': [{'type': 'read'}]}]

In [31]:
# one another way
db.search(User.name.one_of(['jane', 'John']))

[{'name': 'John', 'age': 65}]